# Translating Family Relations

This notebook explores how GLIF can support GF-based translation by discarding contradictory readings.
Concretely, we will look at the example sentence

***Kim is Ahmed's cousin and the father of Grace.***

The source of ambiguity is that *cousin* could refer to either a male or a female cousin.
In German, there are different words for a male cousin (*Cousin*) and a female cousin (*Cousine*).
(Technically, you could argue that *Cousin* and *Cousine* are the masculine and feminine form of the same
word but then this example wouldn't work. Instead, we will treat *Cousin* and *Cousine* as different words,
like e.g. *brother* and *sister* would be treated as different words.)

Parsing the English sentence with GF, we would get two ASTs and thus two translations (one with *Cousin* and one with *Cousine*).
However, we know that Kim is also the father of Grace and therefore Kim has to be male.
Using GLIF, we can filter out readings with inconcistent gender assignments.

In [1]:
archive tmpGLIF/examples relatives

Successfully changed archive

## Grammar

We'll create a small example grammar with concrete syntaxes for German and English.
There is nothing really interesting happening here.

In [2]:
abstract Grammar = {
    cat
        Person;
        Persons;            -- can be object but not subject
        Relation;           -- sibling
        FilledRelation;     -- Jane's cousin,  the father of Ahmed
        S;                  -- Sentence
    fun
        persons : Person -> Persons;
        andPers : Person -> Persons -> Persons;
        andFRel : FilledRelation -> FilledRelation -> FilledRelation;
        fill : Relation -> Persons -> FilledRelation;
        mkS : Person -> FilledRelation -> S;
        
        kim, ahmed, grace : Person;
        father, mother, cousin_male, cousin_female, sister, brother : Relation;
}

Successfully imported Grammar.gf

In [3]:
concrete GrammarEng of Grammar = {
    oper
        mkPers : Str -> Person = \s -> lin Person {nom = s; gen = s + "'s"};
    lincat
        Person = {nom : Str ; gen : Str};
        Persons = {nom : Str ; gen : Str};
        Relation = {of_ : Str ; gen : Str};
        FilledRelation = Str;
        S = Str;
    lin
        persons a = a;
        andPers a b = {nom = a.nom ++ "and" ++ b.nom; gen = a.gen ++ "and" ++ b.gen};
        andFRel a b = a ++ "and" ++ b;
        fill r p = (p.gen ++ r.gen) | (r.of_ ++ "of" ++ p.nom);
        mkS p r = p.nom ++ "is" ++ r;
        
        kim = mkPers "Kim";
        ahmed = mkPers "Ahmed";
        grace = mkPers "Grace";
        father = {of_ = "the father"; gen = "father"};
        mother = {of_ = "the mother"; gen = "mother"};
        cousin_male = {of_ = "a cousin"; gen = "cousin"};
        cousin_female = {of_ = "a cousin"; gen = "cousin"};
        sister = {of_ = "a sister"; gen = "sister"};
        brother = {of_ = "a brother"; gen = "brother"};
}

Successfully imported GrammarEng.gf

In [4]:
concrete GrammarGer of Grammar = {
    oper
        mkPers : Str -> Person = \s -> lin Person {nom = s; gen = s + "s"};
    lincat
        Person = {nom : Str ; gen : Str};
        Persons = {nom : Str ; gen : Str};
        Relation = {of_ : Str ; gen : Str};
        FilledRelation = Str;
        S = Str;
    lin
        persons a = a;
        andPers a b = {nom = a.nom ++ "und" ++ b.nom; gen = a.gen ++ "und" ++ b.gen};
        andFRel a b = a ++ "und" ++ b;
        fill r p = (p.gen ++ r.gen) | (r.of_ ++ "von" ++ p.nom);
        mkS p r = p.nom ++ "ist" ++ r;
        
        kim = mkPers "Kim";
        ahmed = mkPers "Ahmed";
        grace = mkPers "Grace";
        father = {of_ = "der Vater"; gen = "Vater"};
        mother = {of_ = "die Mutter"; gen = "Mutter"};
        cousin_male = {of_ = "ein Cousin"; gen = "Cousin"};
        cousin_female = {of_ = "eine Cousine"; gen = "Cousine"};
        sister = {of_ = "eine Schwester"; gen = "Schwester"};
        brother = {of_ = "ein Bruder"; gen = "Bruder"};
}

Successfully imported GrammarGer.gf

In [5]:
-- Parsing gives us two ASTs
p -lang=Eng "Kim is Ahmed's cousin and the father of Grace"
-- And subsequent linearization gives us two translations
p -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | l -lang=Ger

mkS kim (andFRel (fill cousin_female (persons ahmed)) (fill father (persons grace))) mkS kim (andFRel (fill cousin_male (persons ahmed)) (fill father (persons grace)))

Kim ist Ahmeds Cousine und Graces Vater Kim ist Ahmeds Cousin und Graces Vater

## Semantics Construction

We create a very simply logic and calculus for finding contradictions.
The semantics extraction only extracts gender information and discards everything else.

In [6]:
// The syntax of our logic is very simple ❚
theory logic : ur:?LF =
    proposition : type ❘ # o ❙
    pers : type ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 prec 20 ❙
    not : o ⟶ o ❘ # ¬ 1 prec 30 ❙
❚

Successfully imported logic.mmt

In [7]:
// The discourse domain theory (DDT) introduces e.g. the predicates and constants we will need. ❚
theory DDT : ?logic =
    fem : pers ⟶ o ❙
    male : pers ⟶ o ❙
    kim : pers ❙
    ahmed : pers ❙
    grace : pers ❙
    true : o ❙
    
    // Create dummy type for things that will be discarded during β-reduction ❙
    dummytype : type ❙
    dummy : dummytype ❙
❚

Successfully imported DDT.mmt

In [8]:
theory calculus : ur:?LF =
    include ?logic ❙
    ded : o ⟶ type ❘ # ⊢ 1 prec 5 ❘ role Judgment ❙  // You can read "⊢A" as "A is true" ❙
    contradiction : type ❘ # ↯ ❘ role Judgment ❙
    findContra : {A : o} ⊢ A ⟶ ⊢ ¬A ⟶ ↯ ❙            // if both A and ¬A are true, we have a contradiction ❙
    // for our example, we only need rules for conjunction elimination ❙
    andEl : {A} {B} ⊢ A ∧ B ⟶ ⊢ A ❙
    andEr : {A} {B} ⊢ A ∧ B ⟶ ⊢ B ❙
    
    // domain specific rules ❙
    include ?DDT ❙
    genderContra : {p : pers} ⊢(male p) ⟶ ⊢(fem p) ⟶ ↯ ❙
❚

Successfully imported calculus.mmt

In [9]:
view GrammarSemantics : http://mathhub.info/tmpGLIF/examples/relatives/Grammar.gf?Grammar -> ?DDT =
    Person = pers ❙
    Persons = dummytype ❙
    Relation = pers ⟶ o ❙
    FilledRelation = pers ⟶ o ❙
    S = o ❙

    persons = [x] dummy ❙
    andPers = [x,y] dummy ❙
    andFRel = [p,q] [x] (and (p x) (q x)) ❙
    fill = [p,x] p ❙
    mkS = [x,p] (p x) ❙
    kim = kim ❙
    ahmed = ahmed ❙
    grace = grace ❙
    father = [x : pers] (male x) ❙
    mother = [x] (fem x) ❙
    cousin_male = [x] (male x) ❙
    cousin_female = [x] (fem x) ❙
    sister = [x] (fem x) ❙
    brother = [x] (male x) ❙
❚

Successfully imported GrammarSemantics.mmt

### Testing the Semantics Construction

In [10]:
parse -lang=Eng "Kim is the father of Grace"

mkS kim (fill father (persons grace))

In [11]:
-- semantics construction results in a lot of dummy constants
-- but they get removed by β-reduction (if we don't suppress the simplification)
parse -lang=Eng "Kim is the father of Grace" | construct -no-simplify
parse -lang=Eng "Kim is the father of Grace" | construct

([x,p]p x) kim (([p,x]p) ([x:pers]male x) (([x]dummy) grace))

male kim

In [12]:
-- the example sentence results in two semantic expressions
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | construct

(fem kim)∧(male kim) (male kim)∧(male kim)

## Semantic/Pragmatic Analysis

Using a prover generated from the `calculus` theory, we can discard contradictory readings.

In [13]:
-- generate a prover from the `calculus` theory
elpigen -mode=simpleprover calculus
-- generate signature of DDT
elpigen DDT

Successfully created calculus.elpi

Successfully created DDT.elpi

In [14]:
elpi-notc: checker
accumulate calculus.  % generated prover
accumulate Grammar.   % signature of ASTs (we don't use them here)
accumulate DDT.       % signature of discourse domain theory

% The `check` predicate fails if the prover found a contradiction (using iterative deepening up to depth 7)
check Item :- glif.getLog Item P, ded/hyp _ P => contradiction (idcert 7), !, fail.
check _.

Successfully imported checker.elpi checker.elpi is the new default file for ELPI commands

## Final Demo

In [15]:
-- we get 2 ASTs
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace"

mkS kim (andFRel (fill cousin_female (persons ahmed)) (fill father (persons grace))) mkS kim (andFRel (fill cousin_male (persons ahmed)) (fill father (persons grace)))

In [16]:
-- Semantics construction gives us two logical expressions
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | construct

(fem kim)∧(male kim) (male kim)∧(male kim)

In [17]:
-- Let's filter out all contradictory expressions
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | construct |
    filter -notc -predicate=check

(male kim)∧(male kim)

In [18]:
-- GLIF keeps track of the ASTs, so we can linearize the remaining readings into German:
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | construct |
    filter -notc -predicate=check | linearize -lang=Ger

Kim ist Ahmeds Cousin und Graces Vater

In [19]:
-- With all this effort we removed one of the translations we would have gotten without filtering:
parse -lang=Eng "Kim is Ahmed's cousin and the father of Grace" | linearize -lang=Ger

Kim ist Ahmeds Cousine und Graces Vater Kim ist Ahmeds Cousin und Graces Vater